In [3]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df= spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [6]:
# filter DataFrame for total_votes above or equal to 20
total_votes_df = df.filter(df.total_votes >= 20)
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [7]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes / total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [8]:
# Create paid vine DataFrame
paid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45725380| R84VUCDBCI29U|B0020H46LK|     630657414|        Losing Sleep|           Music|          3|           18|         28|   Y|                N|Suitable for Mass...|Do you still get ...| 2009-07-31|
|         US|   53081008| R4V3ICFDTIDIF|B001SMC91M|     609338227|Tell 'Em What You...|           Music|          4|    

In [9]:
# Create unpaid vine DataFrame
unpaid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   23282003|R2SHXRL6SL1GC9|B010FP0WRU|     974013859|#392: The EP Coll...|           Music|          3|           25|         26|   N|                Y|Up it by a half-s...|A solid collectio...| 2015-08-31|
|         US|   21715314|R2ZC033X86YOY8|B00ZYBH6M0|     408426475|  Live in Tokyo 1975|           Music|          5|    

In [10]:
# Total number of reviews
total_reviews = df.count()
print(f"There are {total_reviews} total reivews")

# Number of 5-star reviews
star5_reviews = df.filter("star_rating = 5").count()
print(f"There are {star5_reviews} 5-star reviews")
print(f"-----------------------------------------------------------------------------")

There are 4751577 total reivews
There are 3290473 5-star reviews
-----------------------------------------------------------------------------


In [11]:
# Percentage of 5-star reviews with vine == 'Y'
total_paid_reviews = paid_df.count()
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
paid_five_star_percent= round((paid_five_star_reviews / total_paid_reviews),2)*100 
print(f"There are total {total_paid_reviews} vine (vine = Y) reviews")
print(f"There are {paid_five_star_reviews},Five-star vine (vine = Y) reviews")
print(f"There are {paid_five_star_percent} % 5-star vine (vine = Y) reviews")
print(f"-----------------------------------------------------------------------------")

There are total 7 vine (vine = Y) reviews
There are 0,Five-star vine (vine = Y) reviews
There are 0.0 % 5-star vine (vine = Y) reviews
-----------------------------------------------------------------------------


In [12]:
# STEP 5 (iv) % of 5-star reviews with vine == 'N'
total_unpaid_reviews = unpaid_df.count()
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100 
print(f"There are total {total_unpaid_reviews} non-vine (vine = N) reviews")
print(f"There are {unpaid_five_star_reviews } 5-star non-vine (vine = N) reviews")
print(f"There are {unpaid_five_star_percent} % 5-star non-vine (vine = N) reviews")
print(f"-----------------------------------------------------------------------------")


There are total 105979 non-vine (vine = N) reviews
There are 67580 5-star non-vine (vine = N) reviews
There are 63.767350135404186 % 5-star non-vine (vine = N) reviews
-----------------------------------------------------------------------------
